In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import afqinsight as afqi
import numpy as np
import seaborn as sns

In [3]:
data = afqi.AFQDataset.from_files(fn_nodes='../data/raw/combined_tract_profiles.csv',
                                  fn_subjects = '../data/raw/participants_updated_id.csv', 
                                  dwi_metrics=['dki_fa', 'dki_md'], target_cols=['age'], index_col="subject_id")

In [4]:
len(data.subjects)

1878

In [5]:
nan_mask = np.isnan(data.y)
y = data.y[~nan_mask]
X = data.X[~nan_mask]
subjects = list(np.array(data.subjects)[~nan_mask])

In [6]:
df = pd.read_csv('../data/raw/participants.csv')
df['subject_id'] = df['subject_id'].apply(lambda s: s.replace('sub-', ''))
df.to_csv('../data/raw/participants_updated_id.csv', index=False)

FileNotFoundError: [Errno 2] No such file or directory: '../data/raw/participants.csv'

In [7]:
# type(data)
data.group_names.index(('dki_md', 'ARC_R'))

25

In [ ]:
data.X.shape

In [ ]:
_ = plt.plot(data.X[:3,2500:2600].T)

In [ ]:
tf_data = data.as_tensorflow_dataset()

In [ ]:
len([x for x in tf_data.as_numpy_iterator()])

In [ ]:
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_validate

In [ ]:
pipeline = Pipeline([('impute', SimpleImputer(strategy='median')),
                     ('scale', StandardScaler()),
                     ('pca', PCA(n_components=100)),
                     ('model', LassoCV())])

In [ ]:
pipeline.fit(X, y)

In [ ]:
pipeline.score(X,y)

In [ ]:
scores = cross_validate(pipeline, X, y)

In [ ]:
scores